In [1]:
!date

Thu May 28 16:16:26 UTC 2020


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
!pip install scvi

**Install packages**

In [0]:
import torch
from scvi.dataset import AnnDatasetFromAnnData, GeneExpressionDataset, SmfishDataset
from scvi.models import VAE, LDVAE
from scvi.inference import UnsupervisedTrainer, load_posterior
from scvi import set_seed

import pandas as pd
import anndata
import scanpy as sc
import numpy as np
import scipy.sparse

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline
sc.set_figure_params(dpi=125)

import seaborn as sns
sns.set(style="whitegrid")

##**Generation of cell atlas**
### Using scVI (LDVAE) for merging of datasets (Lopez et al. 2018)

Read in data

In [0]:
#Stimulation data
path = './drive/My Drive/Colab Notebooks/data/stimulation'
jelly4stim = anndata.read(path + "/jelly4stim_bus_combo_filtered_raw.h5ad")

#Remove Cluster '0' cells
bus_stim = sc.read(path+'/jelly4stim_bus_combo_noZero_filtered_leiden.h5ad')
jelly4stim = jelly4stim[bus_stim.obs_names,]


#Add _ieg to cell names
jelly4stim.obs_names = [i+'_ieg' for i in jelly4stim.obs_names]


stim_data = AnnDatasetFromAnnData(ad = jelly4stim)

#Filter zero count genes
stim_data.filter_genes_by_count(per_batch=True)
print(stim_data)



#Read in Fed/Starved data
path = './drive/My Drive/Colab Notebooks/data/starvation'

oj2 = anndata.read(path + "/bus_fs_combo_filtered_raw.h5ad")

#Add _fs to cell names
oj2.obs_names = [i+'_fs' for i in oj2.obs_names]


fs_data = AnnDatasetFromAnnData(ad = oj2)

#Filter zero count genes
fs_data.filter_genes_by_count(per_batch=True)
print(fs_data)


#Concatenate data (with all nonzero genes)
combo = jelly4stim.concatenate(oj2,join='outer', index_unique=None)

Concatenate data with highly variable genes

In [0]:
all_dataset = GeneExpressionDataset()

stim_data.subsample_genes(5000)
fs_data.subsample_genes(5000)
all_dataset.populate_from_datasets([stim_data,fs_data])

Create and fit LDVAE model

In [0]:
#Set n_latents to 30
vae = LDVAE(
    all_dataset.nb_genes,
    n_batch=all_dataset.n_batches,
    n_latent=30,
    n_layers_encoder=1,
    n_hidden=128,
    reconstruction_loss="nb",
    latent_distribution="normal",
)

In [0]:
trainer = UnsupervisedTrainer(vae,
                              all_dataset,
                              frequency=1,
                              use_cuda=True,
                              n_epochs_kl_warmup=None
                             )

Train model

In [0]:
n_epochs_all = None

n_epochs = 250 if n_epochs_all is None else n_epochs_all
trainer.train(lr=5e-3, n_epochs=250)

In [0]:
#ELBO plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(trainer.history['elbo_train_set'][10:])
ax.plot(trainer.history['elbo_test_set'][10:])

Save data

In [0]:
torch.save(trainer.model.state_dict(),path+'/harmonization.vae.allgenes.model.pkl')

Load data

In [0]:
trainer.model.load_state_dict(torch.load(path+'/harmonization.vae.allgenes.model.pkl'))
#trainer.model.eval()

In [0]:
#Extract latent space
full = trainer.create_posterior(trainer.model, all_dataset, indices=np.arange(len(all_dataset)))
Z_hat = full.sequential().get_latent()[0]
adata = anndata.AnnData(all_dataset.X)
adata.var_names = all_dataset.gene_names
adata.obs_names = list(jelly4stim.obs_names)+list(oj2.obs_names)
adata

Clustering of data

In [0]:
#Use UMAP embedding and Leiden clustering for now
adata.obsm["X_scVI"] = Z_hat
sc.pp.neighbors(adata, use_rep="X_scVI", n_neighbors=1000)
sc.tl.umap(adata)
sc.tl.leiden(adata, key_added="leiden_scVI", resolution=0.8)
adata

In [0]:
sc.pl.umap(adata, color=["leiden_scVI"],color_map='viridis',legend_loc='on data')

In [0]:
#Add labels for experimental origin of each cell
origin_label = list(np.repeat('IEG',len(jelly4stim.obs_names))) + list(np.repeat('FS',len(oj2.obs_names)))
adata.obs['cell_origin'] =  pd.Categorical(origin_label)
adata

Plot latent variables

In [0]:
loadings = vae.get_loadings()
loadings = \
pd.DataFrame.from_records(loadings, index=all_dataset.gene_names,
                                    columns=[f'Z_{i}' for i in range(30)])
loadings.head()

In [0]:
for i, z in enumerate(Z_hat.T):
    adata.obs[f'Z_{i}'] = z

In [0]:
# Plot latent variables
# zs = [f'Z_{i}' for i in range(vae.n_latent)]
# sc.pl.umap(adata, color=zs, ncols=3)

Save adata

In [0]:
# adata.write(path+'/scVI_latent30_combo.h5ad')

# #Save obs to combo data, which contains all nonzero ('unfiltered') genes for downstream analysis/visualizations
# combo.obs['leiden_scVI'] = adata.obs['leiden_scVI']
# combo.obs['cell_origin'] = adata.obs['cell_origin']
# combo.obsm['X_scVI'] = adata.obsm['X_scVI']
# combo.obsm['X_umap'] = adata.obsm['X_umap']

# combo.write(path+'/bus_stim_fs_latent30_combo.h5ad')